In [1]:
from scipy.misc import imread
from preprocess.normalize import preprocess_signature
import tf_signet
from tf_cnn_model import TF_CNNModel
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from find_largest_image import find_largest
import tqdm 

In [2]:
signatures_path = "C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\data\\GPDSSyntheticSignatures4k"
model_path = "C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\models\\signet.pkl"
canvas_size = [1338, 2973] 
print('Using model %s' % model_path)
print('Using canvas size: %s' % (canvas_size,))
sig_df = pd.DataFrame()
# Load the model
model_weight_path = model_path
model = TF_CNNModel(tf_signet, model_weight_path)
image_list = list()
person_list = list()
sig_num_list = list()
fakeness_list = list()
for root, dirs, filenames in tqdm.tqdm(os.walk(signatures_path, topdown=False)):
    if (not root[-1].isdigit()) and not (root == signatures_path) : continue
    for file in filenames:
        if (file.endswith(".jpg")):
            image_list.append(os.path.join(root,file))
            person_list.append(int(root.split('\\')[-1]))
            sig_num_list.append(int(file.split('-')[2].split('.')[0]))
            if (file.startswith('cf')) : fakeness_list.append(1)
            elif (file.startswith('c')) : fakeness_list.append(0)
            else:
                print("invalid character found in image file, quitting!")
                quit() 
image_list = np.asarray(image_list)
person_list = np.asarray(person_list, dtype='uint16')
sig_num_list = np.asarray(sig_num_list, dtype='uint16')
fakeness_list = np.asarray(fakeness_list, dtype='int8')
image_list = np.expand_dims(image_list, axis=1)
person_list = np.expand_dims(person_list, axis=1)
sig_num_list = np.expand_dims(sig_num_list, axis=1)
fakeness_list = np.expand_dims(fakeness_list, axis=1)
image_ndarray = np.zeros((len(image_list), 150, 220),dtype='uint8')
data_ndarray = np.hstack((image_list, person_list, sig_num_list, fakeness_list))

Using model C:\Users\Mert\Documents\GitHub\sigver_bmg\models\signet.pkl
Using canvas size: [1338, 2973]
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.


8002it [00:10, 776.48it/s] 


In [3]:
for counter, full_file_name in enumerate(tqdm.tqdm(image_list[:,0])):
    image_ndarray[counter,:,:] = np.expand_dims(
            preprocess_signature(imread(full_file_name, flatten=True),canvas_size),axis=0)
    # if counter > 3 : break
sess = tf.Session()
sess.run(tf.global_variables_initializer())

splitter_coeff = 100
i_slices = np.split(image_ndarray,splitter_coeff)
vf_slices = np.split(np.zeros((len(image_list),2048)),splitter_coeff)
del image_ndarray, image_list, person_list, sig_num_list, fakeness_list

  0%|                                                                                       | 0/216000 [00:00<?, ?it/s]c:\users\mert\anaconda3\envs\ml_gpu\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
100%|████████████████████████████████████████████████████████████████████████| 216000/216000 [5:50:01<00:00,  6.69it/s]


In [4]:
for c_var, nd_slice in enumerate(tqdm.tqdm(i_slices)):
    feature_vector = model.get_feature_vector_multiple(sess, nd_slice)
    vf_slices[c_var] = feature_vector
sig_v_df = pd.DataFrame(np.vstack(vf_slices))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:53<00:00,  1.01s/it]


In [5]:
del i_slices, vf_slices
sig_d_df = pd.DataFrame(data_ndarray)
sig_d_df.rename(columns={0: 'path', 1: 'person', 2:'sig_num', 3:'fakeness'}, inplace=True)
sig_d_df.to_csv(os.path.join(signatures_path,"data_features.csv"), index=False) 
sig_v_df.to_csv(os.path.join(signatures_path,"visual_features.csv"), index=False)